# YOLOv5 base model tests

In [4]:
# Imports
import os
import json
import shutil
import cv2
import torch
from pathlib import Path

## Model and predictions

In [5]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs


Using cache found in /home/tslermen/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-10-3 torch 1.9.1 CPU

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [6]:
# Image
path = '../../DPVSA_dataset/YOLO_dataset/test/images'
images = []
for img in os.listdir(path):
    f_name = os.path.join(path, img)
    f = cv2.imread(f_name)[..., ::-1]  # OpenCV image (BGR to RGB)
    images.append(f) 

In [7]:
# Inference
results = model(images[:10], size=640)  # includes NMS

# Results
results.print()  
results.show()
# results.save()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

image 1/10: 416x416 18 persons
image 2/10: 416x416 17 persons
image 3/10: 416x416 13 persons
image 4/10: 416x416 12 persons
image 5/10: 416x416 18 persons
image 6/10: 416x416 6 persons
image 7/10: 416x416 8 persons
image 8/10: 416x416 9 persons
image 9/10: 416x416 19 persons, 1 car, 1 truck
image 10/10: 416x416 16 persons
Speed: 24.7ms pre-process, 496.3ms inference, 12.7ms NMS per image at shape (10, 3, 640, 640)


,xmin,ymin,xmax,ymax,confidence,class,name
0,210.338943,357.158478,225.409393,399.987488,0.806097,0,person
1,46.666355,191.692886,54.364338,216.007507,0.671399,0,person
2,278.855011,165.614120,286.387573,188.826340,0.642086,0,person
3,271.739197,230.836105,280.195709,267.202820,0.641802,0,person
4,180.031036,203.513077,188.141388,230.579727,0.620676,0,person
5,367.198456,394.788544,378.668030,416.000000,0.617839,0,person
6,329.827362,170.558929,336.991699,191.697006,0.615442,0,person
7,232.948227,174.684250,239.477264,197.305069,0.558833,0,person
8,303.559753,176.354843,311.480743,208.326767,0.534375,0,person
9,98.152779,144.587524,104.799324,164.282394,0.516932,0,person


## Data preprocessing

In [ ]:
# Change to the dataset root directory
DATASET_ROOT_DIR = Path('../../DPVSA_dataset/dpvsa_dataset/')
YOLO_DATASET_DIR = DATASET_ROOT_DIR / 'YOLO_dataset'


if not os.path.isdir(YOLO_DATASET_DIR):
    os.makedirs(YOLO_DATASET_DIR)
    print("Created folder : ", YOLO_DATASET_DIR)

In [ ]:
def convert_labels_to_yolo_format(file_path):

    """
    Convert the annotation file to the YOLO's format
    'points' will be replaced to [[xc, yc], [width, height]] where
        - xc: bounding box x point
        - yc: bouding box y point
        - width: bounding box width
        - height: bounding box height
    """
    print(file_path)
    with open(file_path) as json_file:
        data = json.load(json_file)
        image_height = data['imageHeight']
        image_width = data['imageWidth']
        
        print(image_width, image_height)
        
        for shape in data['shapes']:
            shape_id = shape['id']
            x1 = shape['points'][0][0]
            x2 = shape['points'][0][1]
            y1 = shape['points'][1][0]
            y2 = shape['points'][1][1]
            shape_width = x2 - x1
            shape_height = y2 - y1
            xc = (x2 - x1)/(2*image_width)
            yc = (y2 - y1)/(2*image_height)
            norm_w = shape_width / image_width
            norm_h = shape_height / image_height
            shape['points'][0][0] = xc
            shape['points'][0][1] = yc
            shape['points'][1][0] = norm_w
            shape['points'][1][1] = norm_h
            print(f'Id: {shape_id}')
            print(f'Points: ({x1}, {y1}), ({x2}, {y2})')
            print(f'Dimensions: {shape_width} x {shape_height}')
            print(f'Normalized dimensions: {norm_w} x {norm_h}')
            print(f'Center: ({xc}, {yc})')
            print(f'New shape: {shape["points"]}')
            break

In [ ]:
def create_yolo_dataset(SOURCE_DATASET_DIR, DESTINATION_DATASET_DIR):
    for i, game in enumerate(os.listdir(SOURCE_DATASET_DIR)):
    
        print(f'Converting game {i} ...')
    
        game_dir = os.path.join(SOURCE_DATASET_DIR, game)
    
        for file in os.listdir(game_dir):
            source = os.path.join(game_dir, file)
        
            if not os.path.isdir(f'{DESTINATION_DATASET_DIR}/train/game{str(i)}'):
                os.makedirs(f'{DESTINATION_DATASET_DIR}/train/game{str(i)}')
            
            destination = os.path.join(f'{DESTINATION_DATASET_DIR}/train/game{str(i)}', file)
        
            if source.endswith('.json'):
                convert_labels_to_yolo_format(source)
            else:
                pass
                #shutil.copy(source, destination)
        break

In [ ]:

create_yolo_dataset(DATASET_ROOT_DIR, YOLO_DATASET_DIR)